# HAGrid, Provisioning and Deployment

In this lesson:

## Deployment Tooling

HAGrid can deploy to a large variety of targets and by leveraging a variety of tools and formats this can be extended. The goal for `hagrid` is to be your friendly helper in navigating the world of deployment and provisioning the `syft` stack and turn a multitude of complex steps and turn them into a handful of uniform and consistent commands.

## HAGrid Installs ON

- macos
- linux
- windows
- wsl2

### Python

We chose to write `HAGrid` as a python cli tool to utilize the existing skills and tooling in our monorepo, however this brings its own problems to the setup process, namely we require:

- Python 3.7
- Console Scripts in `PATH`

Remember in an earlier session when we discussed `PATH`.

### Console Scripts

```python
# packages/hagrid/setup.py

setup(
    name="hagrid",
    description="Happy Automation for Grid",
    long_description="HAGrid is the swiss army knife of OpenMined's PySyft and PyGrid.",
    long_description_content_type="text/plain",
    version=__version__,
    author="Andrew Trask <andrew@openmined.org>",
    packages=find_packages(),
    package_data=DATA_FILES,
    install_requires=packages,
    include_package_data=True,
    entry_points={"console_scripts": ["hagrid = hagrid.cli:cli"]},
)
```

If you notice the part that says `entry_points` this is how you can call a python library as a cli tool. There's just one catch, you need these "console scripts" paths to be setup properly.

This is something which does not always happen automatically with every python install. Currently it appears that installing `python3` with the Windows Store, does not create this entry to the users `PATH` meaning if they install a python library with a cli tool they can't use it without a bunch of manual steps to edit their `PATH`. 

Unfortunately, this means there are many more things that can go wrong with Windows and a higher likelihood that Windows users won't know how to easily solve them.

## Additional Prerequisites

Currently `HAGrid` also requires:
- git
- ansible (for cloud vm deployments)

### Git

During development the fastest way to ship updated code to deployed nodes was simply to utilise the git repository. Of course the repository is large and this adds the requirement for the system to have `git` installed, which is not installed by default on `Windows`.

While we do now have `Continuous Deployment` builds of containers going to `dockerhub` there are several files still required to start the stack, namely the `docker-compose` files and several configuration files which can also require customization.

## Templating

We recently introduced a `manifest` format which will allow us to remove the need to have `git` installed while also providing a single group of minimal files required to start a node. This is something which platforms like `kubernetes` have had for a while in the form of `helm` charts, however no such single format exists for `docker compose` and we suspect that `Docker Desktop` will be a popular container platform for many of our users going forward for some time.

## Bootstrapping Problem

One of the hardest issues when providing software intended to run anywhere and everywhere, is normalizing the environment sufficiently that everything just works the majority of the time.

This is especially hard given both bugs in legacy versions of software that must be supported and in supporting the multitude of updates to all platforms that are generated nearly daily.

Due to all the moving pieces in setting up a `domain` or `network` server, we ended up building a set of tooling to first and foremost help ourselves during development and secondly smooth the setup process for our users who may or may not have the technical know how to configure and deploy container services on their desired target machine.

### Alternatives

Since the majority of what `HAGrid` does is execute system calls in the terminal and parse their output we could certainly switch this something like `go` or `rust` which would enable small binaries without dependencies, at least on initial install, which could greatly ease the setup process.

Alternatively, if we can find a way to run some code first we can attempt to do some setup for users to ease this complication. We have experimented with installer scripts like this:

`win_bootstrap.py`:
https://github.com/OpenMined/PySyft/blob/dev/packages/hagrid/hagrid/win_bootstrap.py

This script detects various prerequisite packages including `choco` package manager, and optionally prompts the user to install them with `choco` even working to manually escalate the user permission prompts as Windows lacks the `sudo` of `POSIX` systems.

Unfortunately this still requires python so for maximum compatibility it will likely need to be re-written in `powershell` as task for which no human deserves.

`install.sh`:  
https://github.com/OpenMined/PySyft/blob/dev/packages/hagrid/scripts/install.sh

Similarly, in the vein of many popular online installers some early experiments with a shell script that can be curled from the web was started. This should provide sufficient control for auto prompt 

## HAGrid Deploys TO

```
docker
azure
gcp
k3d
localhost
x.x.x.x
```

## deploy to `docker`

So far we have mostly dealt with this target since it is the simplest. Remember, HAGrid takes a small number of options and inputs from the user and translates them into lots of command line commands and network calls as needed to setup and provision a node.

In the case of the target `docker` this uses recent versions of `docker` `docker compose` cli tools to do basically everything.

This has a few advantages because `docker` provides a consistent interface across many different oses and platforms and also allows us to mount source code directly into the containers for development.

For some commands we can pass `--cmd=true` and `hagrid` will print a dry run of what it was going to do. This is a handy way to understand what `hagrid` is really doing.

In [7]:
!hagrid launch test_domain domain to docker:8081 --tag=latest --tail=false --dev --cmd=true


⠙ Checking for Docker Service     
✅ Docker service is running
✅ Git 2.37.3
✅ Docker 20.10.17
✅ Docker Compose 2.7.0

                         7*{{{{{sls*{ssc*{{{*{s{s                                  
                       {{{{{*****c<)l|xi<rx{r*{{{{{s                               
                     {{{*sv>\>i\)/;:==v{^;\>+x%cs**{{                              
                   I{{*slv\=;=\l?7yS4XZGdmCl"/"<l{{*{{gRQ                          
                  \ixv)>\/:l#3q4XYU$$UUU$$YXF*l{{{{{{*[yVXPPh                      
                \<>ii)|,` {SXbY&8U8&&&888bggXml>v%%%xclr[#pdEG                     
               ivx%v<^``_{EG&$8UUU8&&&88U&AOkbSr"++//|\>v7FmEEgh                   
            "%%%%>"/=-.+>6bX3uun6hVb8YbXg2pghhPVx'+////")<tEPPgggS                 
           %%%x%%|=+^```%dXe#Sqfnj[h@D83eCL#Fymkg"^/|///\<\?VXEEgE                 
         %%%x%%%%v/+;.. %EVLwj3jTmVk@BD84bddGPPkO!_+|>)"">)<{SXEEEd                
      I{%%x%%%%%%v/+;.,.}XPbkkAOk$YSLTJnd

Lets break up what those commands below are really doing. The first part is a long list of environment variables which are being set before we run our first command.

```bash
RELEASE=development
COMPOSE_DOCKER_CLI_BUILD=1
DOCKER_BUILDKIT=1
HTTP_PORT=8081
HTTPS_PORT=444
TRAEFIK_TAG=51eb25e41d7a4b5351017dcb2feeafcfdde2a743d2407e5b7990b7407e2ea83a
DOMAIN_NAME=test_domain
NODE_TYPE=domain
TRAEFIK_PUBLIC_NETWORK_IS_EXTERNAL=False
VERSION=latest-dev
VERSION_HASH=c53850c31269cd6000afc37c9af92f63c6c4dca2
USE_BLOB_STORAGE=True
STACK_API_KEY=0qkoiw92Ju3jplVcngxbFfj8pjYFBRKeITQW5SCE9YDJOkJd
```

Interesting, we can see that by choosing 8081 as our port that is being reflected in the `HTTP_PORT` environment variable. Additionally, the name of our `domain` is being set, the type of node is `domain` and we seem to have `latest-dev` as our version, due to the `--dev` flag.

If you aren't familiar with environment variables they are ways to expose `key-value` pairs to applications running on operating systems in a way which does not require the os or the application to implicitly define what they are during application compile time.

In `POSIX` operating systems like `linux` and `macOS` these are usually written as `ALL_CAPS=value` by convention. On Windows there are two different syntaxes depending on if you are using `cmd.exe` or `powershell.exe` for powershell it's similar to `POSIX` but with `$env:` prepended like `$env:ALL_CAPS="value"` and some other rules about `SPACING` and `"quotes"` around values.

```
docker compose
-p test_domain
--profile vpn
--profile blob-storage
--profile frontend
--env-file /Users/madhavajay/dev/PySyft/packages/grid/.envfile
--file docker-compose.yml
--file docker-compose.build.yml
--file docker-compose.dev.yml
up -d --build
```

Theres quite a lot going on here so lets unpack it one flag at a time.

We are telling `docker compose` to use `-p` (project name) = `test_domain` which will prefix all the container names on the system. This is particularly useful for running more than one stack for development.

The next three profiles are a way in `docker compose` to toggle containers on and off based on groupings. In this case we are saying this `domain` will have a `vpn`, `frontend` and `blob-storage`. You can see this by checking the compose files themselves.

```yaml
# packages/grid/docker-compose.yml

tailscale:
  profiles:
    - vpn
```

Removing this profile will prevent the deployment of the `tailscale` container for instances where a `firewall` busting `end-to-end` VPN is not desired.

The next line tells `compose` we want to use a custom environment variable file for this deployment where we can set some variables and secrets. This is a duplicate of the defaults in `packages/grid/.env` and the `ENV` variables that `hagrid` generates on your behalf and puts on the command line.

Unfortunately a version of `docker compose` broke command line `ENV` variables and so this workaround is required to maintain compatibility for now. If you check the `hagrid` source you will see how this file is generated on each run.

The next three `--file` directives allow for additional compose `YAML` files to be layered on top of the default one `docker-compose.yml`. The way this works is each successive file overwrites all keys that it defines. In this instance because we are using `dev` mode we need `docker-compose.build.yml` and `docker-compose.dev.yml` so help compose know how to build the containers with our local code and how to mount the source code and configuration files for `hot-reloading` and open additional ports.

### Hot Reloading

Let's see how hot reloading of the `backend` and `syft` is achieved and what some of the implications of it are.

If you look closely at the `backend.dockerfile` you will notice that we install `syft` in editable mode. This is so that modifications to the `/app/syft` path will result in the syft library itself changing.

```dockerfile
# packages/grid/backend/backend.dockerfile

...

# install syft
RUN --mount=type=cache,target=/root/.cache \
  pip install --user -e /app/syft
```

Additionally during this special `docker-compose.dev.yml` file we can then overwrite those internal paths in the container with volume mounts of our local `PySyft` code repo.

```yml
# packages/grid/docker-compose.dev.yml

backend:
    volumes:
      - ${RELATIVE_PATH}./backend/grid:/app/grid
      - ${RELATIVE_PATH}./backend/alembic:/app/alembic
      - ${RELATIVE_PATH}../syft:/app/syft
      - ${RELATIVE_PATH}./data/package-cache:/root/.cache
    command: /start-reload.sh
    environment:
      - PROFILE=true
      - JAEGER_HOST=docker-host
      - JAEGER_PORT=6831
```

We then also replace the `command` with a version which starts our web server with `hot-reloading` using `exec uvicorn --reload`.

The final part of the `compose` command is `up -d --build`, where we ask compose to start our stack in `-d` detached mode and force a `--build` since the code may have changed in `dev` mode.

## deploy to `custom ip`

Let's look at a different example to understand a bit more about what `hagrid` can do. What if we don't want to deploy to our `Desktop` machine but somewhere else?

Well if that `somewhere else` is an existing `linux` server somewhere that supports `ssh` to connect to and issue commands at we can ask `hagrid` to set it up for us.

The way this works is by leveraging a tool called `ansible` which allows us to define the state in which we want a system to be in and have it run commands called a `playbook` against that machine until it gets into the desired state.

`Ansible` is only available on `linux` and `macos` but it should be automatically installed with `hagrid` if you are on those operating systems. If you are on `Windows` you could try installing `hagrid` into `wsl2` which should provide ansible.

Assuming we have the `IP`, and we have login credentials such as a `password` or `ssh-key` for a user on the machine with elevated priviledges we can run:

In [8]:
!hagrid launch test_domain domain to 100.0.0.1 --tag=latest --tail=false --cmd=true


Username for 100.0.0.1 with sudo privledges? [azureuser]: ^C
Error: 




```
cd /Users/madhavajay/dev/PySyft/packages/grid;ANSIBLE_CONFIG=/Users/madhavajay/dev/PySyft/packages/grid/ansible.cfg ansible-playbook -i 100.0.0.1, /Users/madhavajay/dev/PySyft/packages/grid/ansible/site.yml --private-key /Users/madhavajay/.ssh/azureuser.pem --user azureuser -e "node_type='domain'" -e "node_name='test_domain'" -e "github_repo='OpenMined/PySyft'" -e "repo_branch='dev'" -e "docker_tag='latest'" -e "release='production'"
```

This command will prompt us for some remaining settings and finally generate this command to run.

Let's take a look at how the `ansible` command is built up.

```
cd /Users/madhavajay/dev/PySyft/packages/grid;
ANSIBLE_CONFIG=/Users/madhavajay/dev/PySyft/packages/grid/ansible.cfg
```

First we navigate to the `grid` folder and then pass in an `ENV` variable which tells `ansible` to use a specific configuration file.

Next we run `ansible-playbook` against the `IP` we chose with: `ansible-playbook -i 100.0.0.1`. Ansible is used to handling lots of machines at once from an inventory file but here we can tell it how to connect to this one machine.

The playbook path is passed in here `/Users/madhavajay/dev/PySyft/packages/grid/ansible/site.yml` which provides the instructions on what to do via a series of `roles`.

In the case of `ssh-key` we pass in the path to it under `--private-key` and the user to log in with as `--user`.

Finally, a series of `key-value` vars are passed in with the `-e "key='value'"` flag allowing us to override defaults found in the `vars.yml` file at `packages/grid/ansible/group_vars/all/vars.yml`.

Assuming the `credentials` match ansible will log in and get to work.

The `site.yml` file above lists a handful of roles which the target machine will be setup for.

```yml
# packages/grid/ansible/site.yml

---
- hosts: all
  gather_facts: False
  environment:
    LC_ALL: en_US.UTF-8
  become: yes
  roles:
    - node
    - jupyter
    - containers
    - update
```

Here we can see, `node`, `jupyter`, `containers` and `update`.

If we check the folder structure of `packages/ansible/roles`` we should see:

```
├── containers
│   ├── handlers
│   │   └── main.yml
│   └── tasks
│       ├── containers.yml
│       ├── hagrid.yml
│       ├── main.yml
│       ├── src.yml
│       └── tls.yml
├── jupyter
│   └── tasks
│       └── main.yml
├── networkm
│   └── tasks
│       └── main.yml
├── node
│   ├── handlers
│   │   └── main.yml
│   └── tasks
│       ├── docker.yml
│       ├── main.yml
│       ├── security.yml
│       └── system.yml
└── update
    └── tasks
        └── main.yml
```

Ansible is about convention over configuration so each of those `roles` listed above will match a folder name and inside that folder will be a `tasks` folder with a `main.yml` which will get executed.

```yml
# packages/grid/ansible/roles/node/tasks/main.yml
---
...

- name: Install docker
  import_tasks: docker.yml
  when: deploy_only is not defined and install == "true"
```

Here we can see that one of these tasks is to import and run another task for installing `docker`.

### Ansible Modules

One of the beautiful things about `ansible` is that while this has been mostly built and tested for `Ubuntu 20.04` it will be trivial to add support for most other `linux` operating systems as many of the ansible modules already support them, automatically substituting package managers and service runners for us with minimal changes.

## deploy to `azure`

## Desktop Deployments

The important distinctions when considering if a deployment is desktop or not is really related to who is deploying a node, which machine they are running `hagrid` on and what the `os` and `container` host will be.

- MacOS  
  Docker Desktop

- Windows  
  Docker Desktop

- Windows WSL2  
  Docker Engine

- Linux  
  Docker Engine

Of course other combinations are possible such as running the linux version of `Docker Desktop` on `linux` or possibly even `wsl2` but the principal remains the same.

## Server Deployments

In [1]:
## Cake

In [2]:
## Container Hosts

In [ ]:
## 

In [ ]:
how the cli tool works
bootstrapping issues (especially Windows)
python PATH and console-scripts
quickstart
how to use different versions
our target platforms
linux containers
apple silicon linux/arm64
docker
kubernetes
vms on azure and GCP
ansible
packer
building vms
packer
vagrant
cloud deployment
azure
gcp
IP
kubernetes

In [ ]:
Homework
test out quickstart
try a bunch of tutorials?